# Model 0 : Naive Bayes (baseline), this is from sklearn ML map

In [1]:
from core.data_sets import train_sentences, train_labels, val_labels,val_sentences
from models.base_model import model_0
from models.model1 import model_1
from core.helper_fuctions import calculate_results


In [2]:
model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [3]:
# Evaluate our baseline model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achiveves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achiveves an accuracy of: 79.27%


In [4]:
baseline_preds = model_0.predict(val_sentences)
baseline_results  = calculate_results(y_true=val_labels, y_pred=baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

# Model 1: A simple dense model

In [5]:
import tensorflow as tf
from core.helper_fuctions import create_tensorboard_callback


In [6]:
train_sentences.shape, train_labels.shape, val_sentences.shape, val_labels.shape

((6851,), (6851,), (762,), (762,))

In [7]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [8]:
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(), metrics=["accuracy"])

model_1_history = model_1.fit(x=train_sentences, y=train_labels, validation_data=(val_sentences, val_labels), epochs=5, callbacks=[
                              create_tensorboard_callback(dir_name="model_logs", experiment_name="model_1_dense")])

# model_1_history = model_1.fit(x=train_sentences, y=tf.expand_dims(train_labels, axis=-1), validation_data=(val_sentences, tf.expand_dims(val_labels, axis=-1)),epochs=5, callbacks=[
#                               create_tensorboard_callback(dir_name="model_logs", experiment_name="model_1_dense")])



Saving TensorBoard log files to: model_logs/model_1_dense/20230103-005823
Epoch 1/5
215/215 [==============================] - 3s 7ms/step - loss: 0.6105 - accuracy: 0.6933 - val_loss: 0.5339 - val_accuracy: 0.7625
Epoch 2/5
215/215 [==============================] - 1s 6ms/step - loss: 0.4403 - accuracy: 0.8167 - val_loss: 0.4732 - val_accuracy: 0.7822
Epoch 3/5
215/215 [==============================] - 1s 6ms/step - loss: 0.3455 - accuracy: 0.8643 - val_loss: 0.4592 - val_accuracy: 0.7874
Epoch 4/5
215/215 [==============================] - 1s 6ms/step - loss: 0.2841 - accuracy: 0.8908 - val_loss: 0.4629 - val_accuracy: 0.7822
Epoch 5/5
215/215 [==============================] - 1s 7ms/step - loss: 0.2365 - accuracy: 0.9124 - val_loss: 0.4864 - val_accuracy: 0.7822


In [9]:
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.4864 - accuracy: 0.7822


[0.4863888919353485, 0.7821522355079651]

In [10]:
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs.shape


24/24 [==============================] - 0s 3ms/step


(762, 1)

In [11]:
model_1_preds =  tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [12]:
model_1_results = calculate_results(y_true=val_labels, y_pred=model_1_preds)
model_1_results

{'accuracy': 78.21522309711287,
 'precision': 0.7872762884426117,
 'recall': 0.7821522309711286,
 'f1': 0.7789371307867892}

# LSTM Model

In [13]:
from models.model2 import model_2
import tensorflow as tf
from core.helper_fuctions import create_tensorboard_callback


In [14]:
model_2.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.Adam(), metrics=["accuracy"])

model_2_history = model_2.fit(train_sentences, train_labels, epochs=5, validation_data=(val_sentences, val_labels), callbacks=[create_tensorboard_callback("model_logs", "model_2_LSTM")])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20230103-005833
Epoch 1/5
215/215 [==============================] - 8s 19ms/step - loss: 0.2167 - accuracy: 0.9235 - val_loss: 0.6272 - val_accuracy: 0.7717
Epoch 2/5
215/215 [==============================] - 3s 15ms/step - loss: 0.1577 - accuracy: 0.9407 - val_loss: 0.6249 - val_accuracy: 0.7795
Epoch 3/5
215/215 [==============================] - 3s 15ms/step - loss: 0.1286 - accuracy: 0.9514 - val_loss: 0.7422 - val_accuracy: 0.7822
Epoch 4/5
215/215 [==============================] - 3s 14ms/step - loss: 0.1062 - accuracy: 0.9588 - val_loss: 0.9035 - val_accuracy: 0.7756
Epoch 5/5
215/215 [==============================] - 3s 14ms/step - loss: 0.0809 - accuracy: 0.9677 - val_loss: 0.8981 - val_accuracy: 0.7690


In [15]:
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

24/24 [==============================] - 1s 5ms/step


array([[1.1485800e-02],
       [8.0460447e-01],
       [9.9981660e-01],
       [6.1161280e-02],
       [2.3012111e-04],
       [9.9727839e-01],
       [8.1561399e-01],
       [9.9997306e-01],
       [9.9990952e-01],
       [3.7310097e-01]], dtype=float32)

In [16]:
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))

model_2_results = calculate_results(y_true=val_labels, y_pred=model_2_preds)
model_2_results

{'accuracy': 76.9028871391076,
 'precision': 0.7731540113429877,
 'recall': 0.7690288713910761,
 'f1': 0.7657803921853192}

# GRU

In [17]:
from models.model3 import model_3

In [18]:
model_3.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=tf.keras.optimizers.Adam(), metrics=[tf.keras.metrics.Accuracy()])

model_3_histroy = model_3.fit(train_sentences, train_labels, epochs=5, validation_data=(val_sentences,val_labels), callbacks=[create_tensorboard_callback("model_logs","model_3_GRU")])

Saving TensorBoard log files to: model_logs/model_3_GRU/20230103-005854
Epoch 1/5
215/215 [==============================] - 5s 14ms/step - loss: 0.1618 - accuracy: 0.0000e+00 - val_loss: 0.8362 - val_accuracy: 0.0000e+00
Epoch 2/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0848 - accuracy: 0.0000e+00 - val_loss: 0.8286 - val_accuracy: 0.0000e+00
Epoch 3/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0706 - accuracy: 0.0000e+00 - val_loss: 0.8380 - val_accuracy: 0.0000e+00
Epoch 4/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0607 - accuracy: 0.0000e+00 - val_loss: 1.0676 - val_accuracy: 0.0000e+00
Epoch 5/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0540 - accuracy: 0.0000e+00 - val_loss: 1.1418 - val_accuracy: 0.0000e+00


In [19]:
from core.helper_functions2 import process_the_model_output

process_the_model_output(tf, model_3, val_sentences, val_labels, calculate_results)


24/24 [==============================] - 0s 5ms/step - loss: 1.1418 - accuracy: 0.0000e+00
Model Evaluation result [1.1417713165283203, 0.0]
24/24 [==============================] - 0s 4ms/step


{'accuracy': 76.64041994750657,
 'precision': 0.7670590562420062,
 'recall': 0.7664041994750657,
 'f1': 0.764784113056577}

In [20]:
process_the_model_output(tf, model_2, val_sentences, val_labels, calculate_results)


24/24 [==============================] - 0s 7ms/step - loss: 1.0553 - accuracy: 0.7585
Model Evaluation result [1.0553150177001953, 0.7585301995277405]
24/24 [==============================] - 0s 5ms/step


{'accuracy': 75.8530183727034,
 'precision': 0.7592089134633101,
 'recall': 0.7585301837270341,
 'f1': 0.7567321197360164}

# Bidirectional

In [21]:
import tensorflow as tf
from core.data_sets import train_sentences, train_labels, val_labels, val_sentences
from core.helper_fuctions import calculate_results
from models.model4 import model_4
from core.helper_functions2 import process_the_model_output

from core.helper_fuctions import create_tensorboard_callback


In [22]:
model_4.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.Adam(), metrics=["accuracy"])

model_4_history = model_4.fit(train_sentences, train_labels, epochs=5, validation_data=(val_sentences,val_labels), callbacks=[create_tensorboard_callback("model_logs", "model_4_bidirectional")])

process_the_model_output(tf, model_4, val_sentences,
                         val_labels, calculate_results)


Saving TensorBoard log files to: model_logs/model_4_bidirectional/20230103-005910
Epoch 1/5
215/215 [==============================] - 13s 30ms/step - loss: 0.1002 - accuracy: 0.9677 - val_loss: 0.9552 - val_accuracy: 0.7572
Epoch 2/5
215/215 [==============================] - 5s 22ms/step - loss: 0.0574 - accuracy: 0.9739 - val_loss: 1.2673 - val_accuracy: 0.7690
Epoch 3/5
215/215 [==============================] - 5s 22ms/step - loss: 0.0438 - accuracy: 0.9790 - val_loss: 1.4234 - val_accuracy: 0.7559
Epoch 4/5
215/215 [==============================] - 5s 23ms/step - loss: 0.0441 - accuracy: 0.9791 - val_loss: 1.4472 - val_accuracy: 0.7585
Epoch 5/5
24/24 [==============================] - 0s 10ms/step - loss: 1.5146 - accuracy: 0.7546
Model Evaluation result [1.5146050453186035, 0.7545931935310364]
24/24 [==============================] - 2s 8ms/step


{'accuracy': 75.45931758530183,
 'precision': 0.7543557140594386,
 'recall': 0.7545931758530183,
 'f1': 0.7544418225373266}

In [23]:
model_4.evaluate(val_sentences, val_labels)


24/24 [==============================] - 0s 10ms/step - loss: 1.5146 - accuracy: 0.7546


[1.5146050453186035, 0.7545931935310364]

# Conv1D

In [24]:
import tensorflow as tf
from core.data_sets import train_sentences, train_labels, val_labels, val_sentences
from core.helper_fuctions import calculate_results
from models.model5 import model_5
from core.helper_functions2 import process_the_model_output

from core.helper_fuctions import create_tensorboard_callback


In [25]:
model_5.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(), metrics=["accuracy"])

model_5_history = model_5.fit(train_sentences, train_labels, epochs=5, validation_data=(
    val_sentences, val_labels), callbacks=[create_tensorboard_callback("model_logs", "model_5_convulutional")])

process_the_model_output(tf, model_5, val_sentences,
                         val_labels, calculate_results)


Saving TensorBoard log files to: model_logs/model_5_convulutional/20230103-005945
Epoch 1/5
215/215 [==============================] - 9s 10ms/step - loss: 0.1220 - accuracy: 0.9568 - val_loss: 0.9840 - val_accuracy: 0.7651
Epoch 2/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0767 - accuracy: 0.9710 - val_loss: 1.0716 - val_accuracy: 0.7572
Epoch 3/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0607 - accuracy: 0.9769 - val_loss: 1.1408 - val_accuracy: 0.7585
Epoch 4/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0538 - accuracy: 0.9793 - val_loss: 1.2193 - val_accuracy: 0.7546
Epoch 5/5
24/24 [==============================] - 0s 4ms/step - loss: 1.2292 - accuracy: 0.7559
Model Evaluation result [1.229182481765747, 0.7559055089950562]
24/24 [==============================] - 0s 3ms/step


{'accuracy': 75.59055118110236,
 'precision': 0.7556483058194802,
 'recall': 0.7559055118110236,
 'f1': 0.7548850741589771}

# Pretrained

In [26]:
import tensorflow as tf
from core.data_sets import train_sentences, train_labels, val_labels, val_sentences
from core.helper_fuctions import calculate_results
from models.model6 import model_6
from core.helper_functions2 import process_the_model_output

from core.helper_fuctions import create_tensorboard_callback
import tensorflow_hub as hub


In [27]:
model_6.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(), metrics=["accuracy"])

model_6_history = model_6.fit(train_sentences, train_labels, epochs=5, validation_data=(
    val_sentences, val_labels), callbacks=[create_tensorboard_callback("model_logs", "model_6_transferlearning")])

process_the_model_output(tf, model_6, val_sentences,
                         val_labels, calculate_results)


Saving TensorBoard log files to: model_logs/model_6_transferlearning/20230103-010012
Epoch 1/5
215/215 [==============================] - 10s 25ms/step - loss: 0.5019 - accuracy: 0.7841 - val_loss: 0.4458 - val_accuracy: 0.8045
Epoch 2/5
215/215 [==============================] - 5s 23ms/step - loss: 0.4138 - accuracy: 0.8146 - val_loss: 0.4363 - val_accuracy: 0.8058
Epoch 3/5
215/215 [==============================] - 5s 23ms/step - loss: 0.4007 - accuracy: 0.8192 - val_loss: 0.4341 - val_accuracy: 0.8123
Epoch 4/5
215/215 [==============================] - 5s 24ms/step - loss: 0.3920 - accuracy: 0.8247 - val_loss: 0.4298 - val_accuracy: 0.8123
Epoch 5/5
24/24 [==============================] - 1s 21ms/step - loss: 0.4265 - accuracy: 0.8084
Model Evaluation result [0.4264826774597168, 0.808398962020874]
24/24 [==============================] - 1s 21ms/step


{'accuracy': 80.83989501312337,
 'precision': 0.8082433377531367,
 'recall': 0.8083989501312336,
 'f1': 0.807960943828216}